<a href="https://colab.research.google.com/github/Mimitabf/CNAM-RCP216/blob/main/acpTP_et_devoir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TP ACP
**bold text**

In [1]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples').getOrCreate()
spark

In [6]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType
spamfields= [StructField("val"+str(i),DoubleType(),True) for i in range(0,57)]
spamSchema=StructType(spamfields).add("label",DoubleType(),True)
spamDF=spark.read.format("csv").schema(spamSchema).load("spambase.data").cache()
spamDF.printSchema()

root
 |-- val0: double (nullable = true)
 |-- val1: double (nullable = true)
 |-- val2: double (nullable = true)
 |-- val3: double (nullable = true)
 |-- val4: double (nullable = true)
 |-- val5: double (nullable = true)
 |-- val6: double (nullable = true)
 |-- val7: double (nullable = true)
 |-- val8: double (nullable = true)
 |-- val9: double (nullable = true)
 |-- val10: double (nullable = true)
 |-- val11: double (nullable = true)
 |-- val12: double (nullable = true)
 |-- val13: double (nullable = true)
 |-- val14: double (nullable = true)
 |-- val15: double (nullable = true)
 |-- val16: double (nullable = true)
 |-- val17: double (nullable = true)
 |-- val18: double (nullable = true)
 |-- val19: double (nullable = true)
 |-- val20: double (nullable = true)
 |-- val21: double (nullable = true)
 |-- val22: double (nullable = true)
 |-- val23: double (nullable = true)
 |-- val24: double (nullable = true)
 |-- val25: double (nullable = true)
 |-- val26: double (nullable = true)
 |-- v

In [7]:
spamDF.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 2788|
|  1.0| 1813|
+-----+-----+



In [8]:
spamEch1 = spamDF.select("val0", "val1", "label").sample(False, 0.5)
print(spamEch1.count())
spamEch2 = spamDF.select("val0", "val1", "label").sample(False, 0.1)
print(spamEch2.count())

2261
424


In [9]:
fractions = {0.0: 0.5, 1.0: 0.5}    # dictionnaire
spamEchStratifie = spamDF.select("val0", "val1", "label").stat.sampleBy("label", fractions, 1)
spamEchStratifie.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 1384|
|  1.0|  907|
+-----+-----+



In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

colsEntree = ["val"+str(i) for i in range(0, 57)]
assembleur = VectorAssembler().setInputCols(colsEntree).setOutputCol("features")
spamDFA = assembleur.transform(spamDF)
spamDFA.printSchema()
spamDFA.select("features").show(3)


root
 |-- val0: double (nullable = true)
 |-- val1: double (nullable = true)
 |-- val2: double (nullable = true)
 |-- val3: double (nullable = true)
 |-- val4: double (nullable = true)
 |-- val5: double (nullable = true)
 |-- val6: double (nullable = true)
 |-- val7: double (nullable = true)
 |-- val8: double (nullable = true)
 |-- val9: double (nullable = true)
 |-- val10: double (nullable = true)
 |-- val11: double (nullable = true)
 |-- val12: double (nullable = true)
 |-- val13: double (nullable = true)
 |-- val14: double (nullable = true)
 |-- val15: double (nullable = true)
 |-- val16: double (nullable = true)
 |-- val17: double (nullable = true)
 |-- val18: double (nullable = true)
 |-- val19: double (nullable = true)
 |-- val20: double (nullable = true)
 |-- val21: double (nullable = true)
 |-- val22: double (nullable = true)
 |-- val23: double (nullable = true)
 |-- val24: double (nullable = true)
 |-- val25: double (nullable = true)
 |-- val26: double (nullable = true)
 |-- v

In [19]:
from pyspark.ml.feature import PCA
pcaModel = PCA(k=3, inputCol="features", outputCol="pcaFeatures").fit(spamDFA)
resultat = pcaModel.transform(spamDFA)
print(pcaModel.explainedVariance)
print(pcaModel.pc)

[0.9270270117088842,0.0710429729803354,0.0018437247693527724]
DenseMatrix([[-4.47943057e-05,  3.19661832e-05, -2.51728349e-04],
             [ 4.64073749e-05,  1.03193947e-04,  1.75148807e-05],
             [-6.07026243e-05,  2.42356921e-04, -9.60140867e-04],
             [-4.99124584e-05,  1.02410723e-04,  2.81149957e-04],
             [-5.66182598e-06,  2.30761699e-04, -6.83215305e-04],
             [-3.77491113e-05,  8.61037980e-05,  5.97570275e-04],
             [ 3.19313960e-06,  1.67633961e-04, -1.29369649e-04],
             [-2.68921661e-05,  4.48294221e-05,  9.17711734e-05],
             [-1.14431923e-04,  1.14282957e-04, -3.74300312e-04],
             [-9.49479917e-05,  2.52677623e-04, -6.92213454e-04],
             [-3.82750630e-05,  3.68026096e-05,  6.02149095e-05],
             [-2.90277874e-05,  6.40484922e-05,  7.26895908e-04],
             [-5.12866156e-05, -2.54770554e-05,  3.30931630e-04],
             [-9.17404205e-05, -5.94119723e-05,  1.93296438e-04],
             [

In [20]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",withStd=True, withMean=True)
scalerModel = scaler.fit(spamDFA)
scaledSpamDF = scalerModel.transform(spamDFA)
scaledSpamDF.printSchema()


root
 |-- val0: double (nullable = true)
 |-- val1: double (nullable = true)
 |-- val2: double (nullable = true)
 |-- val3: double (nullable = true)
 |-- val4: double (nullable = true)
 |-- val5: double (nullable = true)
 |-- val6: double (nullable = true)
 |-- val7: double (nullable = true)
 |-- val8: double (nullable = true)
 |-- val9: double (nullable = true)
 |-- val10: double (nullable = true)
 |-- val11: double (nullable = true)
 |-- val12: double (nullable = true)
 |-- val13: double (nullable = true)
 |-- val14: double (nullable = true)
 |-- val15: double (nullable = true)
 |-- val16: double (nullable = true)
 |-- val17: double (nullable = true)
 |-- val18: double (nullable = true)
 |-- val19: double (nullable = true)
 |-- val20: double (nullable = true)
 |-- val21: double (nullable = true)
 |-- val22: double (nullable = true)
 |-- val23: double (nullable = true)
 |-- val24: double (nullable = true)
 |-- val25: double (nullable = true)
 |-- val26: double (nullable = true)
 |-- v

Devoir

1-	Appliquez l’ACP sur le DataFrame scaledSpamDF et mettez les projections des données sur les trois premiers axes principaux dans un DataFrame resultatCR. Examinez les trois plus grandes valeurs propres et comparez-les à celles obtenues précédemment. Que constatez-vous ?


In [23]:
pcaCR = PCA(k=3, inputCol="scaledFeatures", outputCol="pcaCRFeatures").fit(scaledSpamDF)
resultatCR = pcaCR.transform(scaledSpamDF).select("pcaCRFeatures")

print(pcaCR.explainedVariance)
print(pcaModel.explainedVariance)



[0.11564793812358597,0.05732324314509041,0.035143016244656634]
[0.9270270117088842,0.0710429729803354,0.0018437247693527724]


Dans les données centrées et réduites, les trois plus grandes valeurs propres sont plus proches les unes des autres qu'avec les données simplement centrées. Cela reflète les différences importantes de variances initiales entre les variables avant la standardisation.


2-	Appliquez l’ACP non normée sur un échantillon (appelé spamEch3) de 10% du DataFrame spamDF obtenu par échantillonnage simple et mettez les projections des données sur les trois premiers axes principaux dans un DataFrame resultatEch3. Examinez les trois plus grandes valeurs propres et comparez-les à celles obtenues dans l’ACP non normée de spamDF. Que constatez-vous ?



In [26]:
spamEch3 = spamDFA.sample(False, 0.1)
pcaEch3 =  PCA(k=3, inputCol="features", outputCol="pcaFeatures").fit(spamEch3)
resultatEch3 = pcaEch3.transform(spamEch3).select("pcaFeatures")
print(pcaEch3.explainedVariance)

[0.9572287336282095,0.04180831087602708,0.0008484213921503486]


Les valeurs propres calculées à partir d'un échantillon d'environ 10% diffèrent logiquement de celles obtenues sur l'ensemble des données.






3-	appliquer l’ACP sur des données tridimensionnelles obtenues par tirage aléatoire suivant une loi normale isotrope :



In [25]:
from pyspark.sql.functions import randn
from pyspark.ml.feature import PCA

idsDF = spark.range(4601)
idsDF.show(5)

randnDF = idsDF.select("id", randn(seed=1).alias("normale0"),randn(seed=2).alias("normale1"),randn(seed=3).alias("normale2"))

randnDF.describe().show()

#A continuer
# Vecteur
colsEntreeRnd = ["normale"+str(i) for i in range(0, 3)]
assembleurRnd = VectorAssembler().setInputCols(colsEntreeRnd).setOutputCol("features")

# Dataframe
randnDF2 = assembleurRnd.transform(randnDF)
randnDF2.printSchema()

# ACP
pcaRnd = PCA(k=3, inputCol="features", outputCol="pcaRndFeatures").fit(randnDF2)
resRnd = pcaRnd.transform(randnDF2).select("pcaRndFeatures")
pcaRnd.explainedVariance

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+
only showing top 5 rows

+-------+------------------+--------------------+--------------------+-------------------+
|summary|                id|            normale0|            normale1|           normale2|
+-------+------------------+--------------------+--------------------+-------------------+
|  count|              4601|                4601|                4601|               4601|
|   mean|            2300.0|-0.00700368903639...|0.004435365125399705| 0.0120495882028871|
| stddev|1328.3386239961555|  1.0012930374887674|  0.9866736787327006| 0.9851417211661581|
|    min|                 0|  -4.020755311197189|  -4.020755311197189|-3.3261595147017684|
|    max|              4600|   3.956664230398238|   3.956664230398238| 3.0583454168633044|
+-------+------------------+--------------------+--------------------+-------------------+

root
 |-- id: long (nullable = false)
 |-- normale0: double (nullable = false)
 |-- normale1: double 

DenseVector([0.3428, 0.3338, 0.3234])

VP presque égales